This note book train a LSTM model to predict the number of sessions a student will spend in the n_th module based on his/her performance in the previous n-1 modules.

# Prepare Dataset

In [ ]:
# load dataset
!git clone https://github.com/pcarvalh/Self-regulated-spacing-online-class.git

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('/content/Self-regulated-spacing-online-class/processed data/SummarySpacingTime.csv')  

In [ ]:
df

,ds_anon_user_id,module,meantimeToQuiz,nsessions,ndays,npages,nactivities,timeInModule,endToQuiz,meanSessionDiff,quizGrade,pretestGrade,examGrade,Nwords,Nactivities,Npages,crammers
0,Stu_00ea8f20a0942ae1385989b901bec86f,1,969.783773,4,3,27,70,7815.200000,1.833333,155008.333333,10,10.0,27,18754,79,27,Non-Crammer
1,Stu_00ea8f20a0942ae1385989b901bec86f,2,419.983333,5,2,18,69,1089.483333,60.016667,57671.333333,9,10.0,27,9546,74,25,Non-Crammer
2,Stu_00ea8f20a0942ae1385989b901bec86f,3,2429.151982,4,3,39,73,4541.850000,40.700000,44065.333333,10,10.0,27,12785,74,26,Non-Crammer
3,Stu_00ea8f20a0942ae1385989b901bec86f,4,896.453084,2,2,12,51,1499.650000,7.466667,428409.000000,9,10.0,27,9638,63,3,Non-Crammer
4,Stu_00ea8f20a0942ae1385989b901bec86f,5,1090.381925,4,3,20,29,2156.000000,18.366667,251894.200000,10,10.0,27,14163,29,18,Non-Crammer
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6787,Stu_ff9a310ccbfeaadac871b45f8ded4f81,6,233.412032,2,2,19,67,1438.450000,7.283333,37608.000000,9,8.0,22,17373,80,20,Non-Crammer
6788,Stu_ff9a310ccbfeaadac871b45f8ded4f81,7,352.605721,3,2,34,74,3520.166667,8.566667,156778.666667,8,8.0,22,15293,74,10,Non-Crammer
6789,Stu_ff9a310ccbfeaadac871b45f8ded4f81,8,431.174848,4,3,20,39,2095.733333,5.000000,151080.833333,9,8.0,22,11582,39,10,Non-Crammer
6790,Stu_ff9a310ccbfeaadac871b45f8ded4f81,9,772.821111,6,3,23,36,6454.050000,6.666667,185211.750000,8,8.0,22,12430,37,14,Non-Crammer


In [ ]:
df['crammer'] = np.where(df['crammers']=='Non-Crammer', 0, 1)

In [ ]:
df = df.drop(columns=['examGrade', 'crammers'])

In [ ]:
df = df.fillna(value = {'pretestGrade':df['pretestGrade'].mean(), "meanSessionDiff":df["meanSessionDiff"].mean()})

In [ ]:
df['number of sessions' ]= df['nsessions']

In [ ]:
FEATURES = ['meantimeToQuiz', 'npages','nactivities','timeInModule',
            'endToQuiz','meanSessionDiff', 'quizGrade','pretestGrade',
            'Nwords','Nactivities','Npages', 'nsessions']

df['nsessions_normal'] = df['nsessions']

USER_FEATURES = ['meantimeToQuiz',
                'npages','nactivities','timeInModule',
            'endToQuiz','meanSessionDiff', 'quizGrade','pretestGrade',
            'nsessions']

MODULE_FEATURES = ['Nwords','Nactivities','Npages']

In [ ]:
MEAN_ALL = df.mean()
STD_ALL = df.std()
for col in ['meantimeToQuiz', 'npages','nactivities','timeInModule',
            'endToQuiz','meanSessionDiff', 'pretestGrade',
            'Nwords','Nactivities','Npages','ndays','quizGrade']:
  df[col] = (df[col] - MEAN_ALL[col]).div(STD_ALL[col])

In [ ]:
df['nsessions_normal'] = np.log1p(df['nsessions_normal'])
df['nsessions'] = np.log1p(df['nsessions'])

In [ ]:
df

,ds_anon_user_id,module,meantimeToQuiz,nsessions,ndays,npages,nactivities,timeInModule,endToQuiz,meanSessionDiff,quizGrade,pretestGrade,Nwords,Nactivities,Npages,crammer,number of sessions,nsessions_normal
0,Stu_00ea8f20a0942ae1385989b901bec86f,1,-0.333594,1.609438,0.02919,0.053546,1.037562,0.846007,-0.312039,-0.103592,0.931043,-0.265992,1.470412,0.996848,1.319891,0,4,1.609438
1,Stu_00ea8f20a0942ae1385989b901bec86f,2,-0.398759,1.791759,-0.52153,-0.352252,1.001502,-0.647556,-0.305420,-0.484132,0.217475,-0.265992,-0.793981,0.775515,1.080312,0,5,1.791759
2,Stu_00ea8f20a0942ae1385989b901bec86f,3,-0.160620,1.609438,0.02919,0.594611,1.145741,0.119102,-0.307617,-0.537325,0.931043,-0.265992,0.002540,0.775515,1.200101,0,4,1.609438
3,Stu_00ea8f20a0942ae1385989b901bec86f,4,-0.342285,1.098612,-0.52153,-0.622785,0.352427,-0.556471,-0.311398,0.965271,0.217475,-0.265992,-0.771357,0.288581,-1.555061,0,2,1.098612
4,Stu_00ea8f20a0942ae1385989b901bec86f,5,-0.319299,1.609438,0.02919,-0.262075,-0.440888,-0.410718,-0.310158,0.275184,0.931043,-0.265992,0.341412,-1.216487,0.241784,0,4,1.609438
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6787,Stu_ff9a310ccbfeaadac871b45f8ded4f81,6,-0.420873,1.098612,-0.52153,-0.307164,0.929383,-0.570062,-0.311419,-0.562570,0.217475,-0.868229,1.130802,1.041115,0.481363,0,2,1.098612
6788,Stu_ff9a310ccbfeaadac871b45f8ded4f81,7,-0.406746,1.386294,-0.52153,0.369167,1.181801,-0.107780,-0.311273,-0.096671,-0.496094,-0.868229,0.619297,0.775515,-0.716533,0,3,1.386294
6789,Stu_ff9a310ccbfeaadac871b45f8ded4f81,8,-0.397433,1.609438,0.02919,-0.262075,-0.080290,-0.424101,-0.311679,-0.118947,0.217475,-0.868229,-0.293297,-0.773820,-0.716533,0,4,1.609438
6790,Stu_ff9a310ccbfeaadac871b45f8ded4f81,9,-0.356939,1.945910,0.02919,-0.126809,-0.188470,0.543739,-0.311489,0.014489,-0.496094,-0.868229,-0.084760,-0.862353,-0.237375,0,6,1.945910


# LSTM Model

In [ ]:
import tensorflow as tf
import keras
import tensorflow.keras.layers as L
import random
from random import choice
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from keras.models import load_model

tf.config.run_functions_eagerly(True)
random.seed(42)
tf.keras.backend.set_floatx('float64')

In [ ]:
def train_model(X_train, X_test, M_train, M_test, y_train, y_test,n):
  user_input = keras.Input(shape = (n-1,9), name = 'user_input')
  module_input = keras.Input(shape = (n,14), name = 'module_input')
  user_to_lstm1 = L.Dense(64,
                          input_shape = (None, 9),
                          activation = 'relu')
  module_to_lstm1 =  L.Dense(64,
                          input_shape = (None, 14),
                          activation = 'relu')
  user_lstm = L.LSTM(256, input_shape = (None, 64))
  module_lstm = L.LSTM(256, input_shape = (None, 64))
  lstm_to_dense1 =  L.Dense(265,
                          input_shape = (None, 512),
                          activation = 'relu')
  dropout_1 = L.Dropout(0.1)
  dense1_to_dense2 =  L.Dense(128,
                          input_shape = (None, 256),
                          activation = 'relu')
  dense2_to_dense3 =  L.Dense(64,
                          input_shape = (None, 128),
                          activation = 'relu')
  dropout_3 = L.Dropout(0.1)
  dense3_to_dense4 =  L.Dense(32,
                          input_shape = (None, 64),
                          activation = 'relu')
  dense4_to_pred =  L.Dense(1,
                          input_shape = (None, 32),
                          activation = 'linear')

  user_lstm_input = user_to_lstm1(user_input)
  module_lstm_input = module_to_lstm1(module_input)

  user_lstm_output = user_lstm(user_lstm_input)
  module_lstm_output = module_lstm(module_lstm_input)

  # summation of lstm layers outputs
  lstm_output = tf.add(user_lstm_output, module_lstm_output)

  # feed forward neural network
  dense1 = lstm_to_dense1(lstm_output)
  dense2 = dense1_to_dense2(dropout_1(dense1))
  dense3 = dense2_to_dense3(dense2)
  dense4 = dense3_to_dense4(dropout_3(dense3))
  pred = dense4_to_pred(dense4)

  model = keras.Model(
      inputs=[user_input, module_input],
      outputs=[pred],
      name='lstm_model',
  )
  opt_adam = Adam(learning_rate = 0.0001)

  model.compile(
      optimizer=opt_adam,
      loss=  'mse',
      metrics = ['mse', 'mae']
  )

  # we choose the model with best validation loss
  mc = ModelCheckpoint('models/best_model_for_' + str(n) + '.h5', monitor='val_mse', mode='min', verbose=0)
  # early stopping to prevent overfit
  es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=40)

  history = model.fit(
    [X_train, M_train],
    y_train,
    batch_size = 64,
    epochs = 200,
    validation_data=([X_test, M_test], y_test),
    callbacks=[es, mc],
    verbose = 0
  )

  while True:
    try:
      model = load_model('models/best_model_for_' + str(n) + '.h5')
      break
    except FileNotFoundError:
      continue

  return model, history

In [ ]:
users = df['ds_anon_user_id'].unique()

In [ ]:
from sklearn.preprocessing import OneHotEncoder
module_encoder = OneHotEncoder(categories = [[1,2,3,4,5,6,7,8,9,10,11]])

In [ ]:
def process_data(n):
    user_data = []
    module_data = []
    y = []

    for user in users:
      user_df = df[df['ds_anon_user_id'] == user]
      if len(user_df)>=n:
        # Objective: prediction the nsessions on n_th module
        
        # Get module features for first n_th modules
        user_df = user_df.head(n)
        encoded_module = module_encoder.fit_transform(user_df['module'].values.reshape(-1,1)).toarray()
        modules = user_df[MODULE_FEATURES].values

        # our target
        sessions = user_df['nsessions_normal'].values[-1] 
        
        # Get user features for first (n-1) modules
        # ['meantimeToQuiz', 'npages','nactivities','timeInModule',
        #   'endToQuiz','meanSessionDiff', 'quizGrade','pretestGrade',
        #   'nsessions']
        user_df = user_df.head(n-1)
        user = user_df[USER_FEATURES].values

        user_data.append(user)
        module_data.append(np.concatenate((modules, encoded_module), axis = 1))
        y.append([sessions])
      
    return np.array(user_data), np.array(module_data), np.array(y)

In [ ]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

In [ ]:
import math
from sklearn.model_selection import train_test_split
for n in [2, 3, 4, 5, 6, 7, 8, 9, 10, 11]:
  user_data, module_data, y = process_data(n)
  print(y.std())
  X_train, X_test, M_train, M_test, y_train, y_test = train_test_split(user_data, module_data, y, test_size=0.1, random_state=22)
  X_train, X_val, M_train, M_val, y_train, y_val = train_test_split(X_train, M_train, y_train, test_size=int(0.1*len(user_data)), random_state=106)
  
  model, history = train_model(X_train, X_val, M_train, M_val, y_train, y_val, n)
  metrics = model.evaluate([X_test, M_test], y_test, return_dict = False)
  metrics_train = model.evaluate([X_train, M_train], y_train, return_dict = False)

  loss_his = history.history
  mse = min(loss_his['val_mse'])

  print("The metric for ", n, " are: ") 
  print("TRAIN_RMSE: %.3f " % (math.sqrt(metrics_train[1])))
  print("VAL_RMSE: %.3f " % (math.sqrt(mse)))
  print("TEST_RMSE: %.3f " % (math.sqrt(metrics[1])))